<h1 align='center'> CSC4120 Programming Assignment 3  </h1>

## Submission Requirements

   The submission <font color = #FF0000>deadline is March 17 (Sun.), 2024, 11:59 pm</font>. Solutions submitted after the deadline will be graded as 0 points. Please submit a **ipynb** file using given template and clearly state your group members' student IDs. Otherwise, your points will be deducted.

## Student IDs

120040025 - Yohandi

120040007 - Andrew Nathanael


## Remote Error Correction

Prof. Costas was isolated in Xi'an. He was planning to work on the homework during the isolation. Unfortunately, his computer broke and corrupted the latest copy of the homework file. A few lines of the file were affected. Prof. Costas could just ask Mingjie, who was in Shenzhen, to send him the latest copy of the file. But the hotel he stayed has very slow and pricey Internet access.

Help Prof. Costas recover the file and prepare homework on time! Design an interactive protocol for detecting and correcting corrupted lines that uses **little communication**.

In this question, you have to answer two short-answer questions and implement the following three functions in file_compare.ipynb.
(Feel free to add additional methods)

* def hash_function(s)
* def compute_node_value(table, i, j)
* def binary_check_hash(left, right)

First, Prof. Costas and Mingjie need a good hash function that can compute a hash value for any contiguous subset of lines of a file.

1. Implement a **division hash** in the function `hash_function`.

You can use bulit-in function `hash()` (or any other function) to calculate the hash code of a string, but you should make **your** `hash_function` a division hash. 

2. Implement the function `compute_node_value`.

It computes all useful hash values for binary comparison (to be explained later), and store them in corresponding table (`HashCorruptedFile[][]` and `HashOriginalFile[][]`). It only calculates `HashCorruptedFile[0, n]`, `HashCorruptedFile[0, n/2]`, `HashCorruptedFile[0, n/4]`, `HashCorruptedFile[n/2, n]`, `HashCorruptedFile[n/4, n/2]` ... etc.

Hints:
- `wordsCorrupted[i]` := data of line i in the corrupted file
- `HashCorruptedFile[i][i]` := Hash value of "line i in the corrupted File"
- `HashCorruptedFile[i][j]` := Hash value of "lines i to j in the corrupted File"

For example, if the corrupted file (4 lines) is as follows:

```raw
    aaaa\n
    bbbb\n
    cccc\n
    dddd
```

then,
- `wordsCorrupted[0]` = "aaaa"
- `HashCorruptedFile[0][0]` = hash function("aaaa")
- `HashCorruptedFile[1][1]` = hash function("bbbb")
- `HashCorruptedFile[0][2]` = hash function(temp). temp is something related to "aaaa",
"bbbb" and "cccc", or related to their hash value. It depends on your own design.

----


Now Prof. Costas and Mingjie have your hash function. They want to find the corrupted lines. Mingjie came up with a naive algorithm to compare both files line by line (and he wrote the codes in `naive_check_hash`). They could send and compare the hash value of each line. But Prof. Costas is unsatisfied with the communication cost $ \Theta(n) $. Help them design an efficient algorithm! 

3. Suppose the two copies of the file differ in some number of consecutive lines.
Let $ n $ be the total number of lines in the file. How can Prof. Costas and Mingjie use
binary search to find the start and end of the corruption by exchanging only $ O(\log n) $ hash
values? Describe your idea below.

The procedure starts with a comparison of the hash value of the entire range.

- The hash values of the entire files, both original and corrupted, are compared. If these hash values are identical, it implies the absence of corruption.
- If the hash values differ, this indicates the presence of corruption within the file. The file is then divided into two segments: the first half (from start to midpoint) and the second half (from midpoint to end).
    - **Finding the start**: To locate the beginning of the corrupted lines, the hash values of the first half of the file are compared between the original and the corrupted versions. A difference in hash values suggests that the corruption starts within this first half. If no difference is found, the search moves to the second half.
    - **Finding the end**: To determine the ending of the corrupted lines, a similar comparison is conducted starting with the second half first and the first half afterwards (interchange the order to prioritize the rightmost).

The process will be repeated until it finds the smallest part where the original and corrupted file hashes are different. Both start line and end line is computed independently. With that, only $\mathcal{O}(\log n)$ of hash values needed to get the information of start and end lines.

4. Let $ W $ ($ W \ll n $) be the number of corrupted lines (not necessarily consecutive). What can
they do to find all corrupted lines by exchanging only $ O(W · \log n) $ hash values? Describe your idea below.

Assume $W \geq 1$.

- The search begins with the entire file as the initial range $[0, n - 1]$.
- A binary search is conducted within the current range to identify the leftmost corrupted line. This involves comparing hash values of segments within the range, starting with a comparison of the first half of the range. If the first half's hash does not match the expected (original) hash, it indicates the presence of corruption within this half, and the search continues within this segment. Conversely, if the first half's hash matches the original, the search moves to the second half, as the corruption must be located there.
- After identifying a corrupted line at index $i$, the range is narrowed to $[i+1, end]$ for the next iteration, effectively skipping over the identified corrupted line and focusing the search on remaining lines.
- This process is repeated, each time identifying a single corrupted line and then narrowing the search range, until all \(W\) corrupted lines are identified. 

Throughout the process, the binary search prioritizes finding the leftmost corrupted line within the current range. This ensures that only $\mathcal{O}(W)$ searches are performed, maintaining a complexity of  $\mathcal{O}(W \cdot \log n)$.

5. Implement `binary_check_hash`.

This function simulates the interactive protocol (and is the implementation of your idea in 4). It should call the function `compare` to compare two hash values, print `"[binary_check_hash] found damaged content at line xx"` as `naive_check_hash` does whenever a corrupted line is found, and call the function `send_words` to retrieve the original line from the server.

In [9]:

# Hash Values for words in New File
# HashCorruptedFile[i,j] = Hash value of lines i to j in Corrupted File
HashCorruptedFile = []

# Hash Values for words in Old File
# HashOriginalFile[i,j] = Hash value of lines i to j in Original File
HashOriginalFile = []

# Words in the Corrupted and Original File
wordsCorrupted = []
wordsOriginal = []

# n : number of lines in each file
n = 1 # default value

# variable to measure network transmission cost
cost = 0

# penalty value for transmitting the whole line
penalty = 4


# feel free to add additional functions

def hash_function(s):
    '''
    compute and return the hash value of the string s.

    You can use built-in hash() (or any other method) to calculate a hash code. But you should finally return a division hash code (something % something). 
    '''
    divisor = 10**9 + 7
    return hash(s) % divisor

def compute_node_values():
    ''' 
    in the first part of compute_hash_tree(), we computed HashOriginalFile[0][0], HashOriginalFile[1][1], HashOriginalFile[2][2].... etc
    
    Now, here in compute_node_values(), you have to calculate HashOriginalFile[i][j] where i < j
    So, HashOriginalFile[2][4] = hash_function("line 2-4 of corrupted file"). It depends on your own design. 
    
    For example, HashOriginalFile[2][4] can be = hash_function(hash_function(line 2) + 2*hash_function(line 3) + 3*hash_function(line 4)).... but it is good? Perhaps not. So, write your own hash function here! 
    '''
    compute_node_value(HashCorruptedFile, wordsCorrupted, 0, n - 1)
    compute_node_value(HashOriginalFile, wordsOriginal, 0, n - 1)

def compute_node_value(table, words, i, j):
    '''
    it calculates table[i][j], where table could be HashCorruptedFile or HashOriginalFile.

    Remember you are doing a binary search here. It's not necessary to try out all combination of i, j. You only need to consider (0, n), (0, n/2), (n/2, n), ... 
    '''
    if i == j:
        table[i][j] = hash_function(words[i])
    else:
        mid = (i + j) // 2

        if table[i][mid] is None:
            table[i][mid] = compute_node_value(table, words, i, mid)
        if table[mid + 1][j] is None:
            table[mid + 1][j] = compute_node_value(table, words, mid + 1, j)

        table[i][j] = hash_function(str(table[i][mid]) + str(table[mid + 1][j]))

        return table[i][j]


def binary_check_hash(left, right):
    ''' 
    performs binary search over the hash tree.

    Search through the whole tree. 
    The output should be similar to naive_check_hash().
    '''
    global cost, penalty
    if left > right or left < 0 or right >= n:
        return
    
    if left == right:
        if not compare(HashCorruptedFile[left][left], HashOriginalFile[left][left]):
            print("[binary_check_hash] found damaged content at line", left)
            send_word()
        return
    
    mid = (left + right) // 2

    if not compare(hash_function("".join(wordsCorrupted[left:mid+1])), hash_function("".join(wordsOriginal[left:mid+1]))):
        binary_check_hash(left, mid)
    
    if not compare(hash_function("".join(wordsCorrupted[mid+1:right+1])), hash_function("".join(wordsOriginal[mid+1:right+1]))):
        binary_check_hash(mid + 1, right)

# read fileCorrupted and fileOriginal, save to wordsCorrupted and wordsOriginal
def read_files(fileCorrupted, fileOriginal):
    print("reading files:", fileCorrupted, fileOriginal)
    with open(fileCorrupted, 'r') as fCorrupted:
        for line in fCorrupted:
            wordsCorrupted.append(line.rstrip())
    with open(fileOriginal, 'r') as fOriginal:
        for line in fOriginal:
            wordsOriginal.append(line.rstrip())


# Compute the hash values of the leaf nodes in the Hash Tree
def compute_hash_tree():
    for i in range(n):
        HashCorruptedFile.append([])
        HashOriginalFile.append([])
        for _ in range(n):
            HashCorruptedFile[i].append(None)
            HashOriginalFile[i].append(None)

    for i in range(n):
        HashCorruptedFile[i][i] = hash_function(wordsCorrupted[i])
        HashOriginalFile[i][i] = hash_function(wordsOriginal[i])

    compute_node_values()


# compares the two hash values. total cost increase by 1
def compare(oldhash, newhash):
    global cost
    cost += 1
    return oldhash == newhash


# adds the penalty of sending the whole line over network transmission
def send_word():
    global cost, penalty
    cost += penalty

# naive hash check method, goes over all the lines and compares them.
def naive_check_hash(left, right):
    global cost, penalty
    for i in range(left, right + 1):
        if not compare(HashCorruptedFile[i][i], HashOriginalFile[i][i]):
            print("[naive_check_hash] found damaged content at line", i)
            send_word()


# performs the naive_check_hash algorithm and binary_check_hash algorithm.
# compares filename1 and filename2 with n lines each
# print out the cost
def file_transfer(filename1, filename2, n1):
    global cost, n
    n = n1

    print("1. starting computation of hash tree...")
    read_files(filename1, filename2)
    compute_hash_tree()
    print("finished computation of hash tree\n")

    print("2. starting computation of naive_check_hash()...")
    cost = 0
    naive_check_hash(0, n - 1)
    print("cost of navie_check_hash() =", cost, "\n")

    print("3. starting computation of binary_check_hash()...")
    cost = 0
    binary_check_hash(0, n - 1)
    print("cost of binary_check_hash() =", cost, "\n")


if __name__ == "__main__":
    file_transfer("file_corrupted1.txt", "file_original1.txt", 1024)
    file_transfer("file_corrupted2.txt", "file_original2.txt", 4096)
    file_transfer("file_corrupted3.txt", "file_original3.txt", 16384)

1. starting computation of hash tree...
reading files: file_corrupted1.txt file_original1.txt
finished computation of hash tree

2. starting computation of naive_check_hash()...
[naive_check_hash] found damaged content at line 255
[naive_check_hash] found damaged content at line 511
[naive_check_hash] found damaged content at line 767
[naive_check_hash] found damaged content at line 1023
cost of navie_check_hash() = 1040 

3. starting computation of binary_check_hash()...
[binary_check_hash] found damaged content at line 255
[binary_check_hash] found damaged content at line 511
[binary_check_hash] found damaged content at line 767
[binary_check_hash] found damaged content at line 1023
cost of binary_check_hash() = 90 

1. starting computation of hash tree...
reading files: file_corrupted2.txt file_original2.txt
finished computation of hash tree

2. starting computation of naive_check_hash()...
[naive_check_hash] found damaged content at line 255
[naive_check_hash] found damaged content